## **Coleta de dados do Datalake da Basedosdados**
***
##### **Objetivo**: coletar dados sobre série histórica da inflação e venda de gasolina pelos distribuidores, armazenados no data lake público da base dos dados
##### **Resultado**: utilizando-se da biblioteca de autoria da organização base dos dados, utilizei a função que direciona o python, por meio da minha conta no google BigQuery, a executar uma query no data lake da própria organização; por fim, armazenei o resultado de ambas as consultas no banco de dados do projeto ('dados.db').
***

In [1]:
# Bibliotecas utilizadas
import pandas  as pd
import sqlite3 as sql3
import basedosdados

from glob import glob

In [2]:
# Criando / Fazendo conexão com banco de dados do projeto
conn = sql3.connect( '../analise_dados/dados.db' )

### Racional do código

#### 1 Criei uma função que ler arquivos

In [3]:
# Criando função para ler os arquivos '.sql'
def ler_query( arquivo_query ):
    
    # Abrindo arquivo de interesse
    with open( arquivo_query, 'r', encoding = 'utf-8' ) as arquivo:
        
        # Lendo e retornando conteúdo do arquivo
        return arquivo.read()

#### 2 Executando query

##### 2.1 Nome dos arquivos das querys a serem executadas
Coletando e armazenando em uma lista os nomes de arquivos com extensão '.sql' a serem
lidos e executados no datalake

In [4]:
# Coletando nomes
querys = glob( '*.sql' )

querys

['query_gasolina.sql', 'query_inflacao.sql']

##### 2.2 Laço for que itera a lista com nome dos arquivos '.sql'; nele, o arquivo é lido pela função criada e, quando tiver o resultado da consulta, armazenará ele no banco de dados do projeto

In [9]:
for i, item in enumerate( querys ):
    
    # Só há necessidade de autenticação de conta na 1º consulta do BigQuery
    if i == 0: confirmacao = True
    else     : confirmacao = False

    # Consultando BigQuery com query dos arquivos '.sql'
    df = basedosdados.read_sql( ler_query( item ),
                                billing_project_id = 'portfolio-367913',
                                reauth = confirmacao )
    
    # Armazenando dados da consulta no banco de dados do projeto
    df.to_sql( f'{ item[ 6 : -4 ] }',
               con       = conn,
               if_exists = 'replace',
               index     = False )